In [57]:
import pandas as pd
import psycopg2
import json
import os


In [48]:
conn = psycopg2.connect(
    dbname='conversions',
    user='admin',
    password='test',
    host='localhost',
    port=5432
)

#df = pd.read_sql("SELECT * FROM events", conn)

In [49]:
df = pd.read_csv('/home/gladiator1488/conv_tracker/events.csv')
df

,id,client_id,website_id,event_type,timestamp,data
0,835,1753106952170378793,stirka_com_fr,form_field_interaction,2025-07-21 19:13:36.373+03,{}
1,836,1753106952170378793,stirka_com_fr,session_end,2025-07-21 19:13:36.401+03,{}
2,837,1753106952170378793,stirka_com_fr,session_start,2025-07-21 19:13:37.684+03,"{""page_url"": ""https://stirka.com/fr"", ""referre..."
3,838,1753106952170378793,stirka_com_fr,scroll_depth,2025-07-21 19:13:39.845+03,"{""time_on_page"": ""2.16"", ""scroll_percent"": 25}"
4,839,1753106952170378793,stirka_com_fr,scroll_depth,2025-07-21 19:13:40.132+03,"{""time_on_page"": ""2.45"", ""scroll_percent"": 50}"
...,...,...,...,...,...,...
4953,5788,1753522482590609787,stirka_com_fr,click,2025-07-26 12:35:30.431+03,"{""element"": {""id"": null, ""tag"": ""DIV"", ""name"":..."
4954,5789,1753522482590609787,stirka_com_fr,click,2025-07-26 12:35:32.277+03,"{""element"": {""id"": null, ""tag"": ""DIV"", ""name"":..."
4955,5790,1753522482590609787,stirka_com_fr,scroll_depth,2025-07-26 12:35:50.755+03,"{""time_on_page"": ""67.06"", ""scroll_percent"": 50}"
4956,5791,1753522482590609787,stirka_com_fr,scroll_depth,2025-07-26 12:36:17.684+03,"{""time_on_page"": ""93.99"", ""scroll_percent"": 75}"


In [50]:
df = df.drop('website_id', axis=1)
df = df.drop('id', axis=1)


In [51]:
set(df['event_type'].values)

{'click',
 'first_interaction',
 'form_field_interaction',
 'hover',
 'scroll_depth',
 'session_end',
 'session_start',
 'tab_blur_focus'}

In [52]:
len(set(df['client_id'].values))

1112

In [53]:
df['data'] = df['data'].apply(lambda x: json.loads(x) if pd.notnull(x) and x != '' else {})
df_exp = df['data'].apply(pd.Series)
df_new = pd.concat([df.drop(columns='data'), df_exp], axis=1)

In [54]:
df

,client_id,event_type,timestamp,data
0,1753106952170378793,form_field_interaction,2025-07-21 19:13:36.373+03,{}
1,1753106952170378793,session_end,2025-07-21 19:13:36.401+03,{}
2,1753106952170378793,session_start,2025-07-21 19:13:37.684+03,"{'page_url': 'https://stirka.com/fr', 'referre..."
3,1753106952170378793,scroll_depth,2025-07-21 19:13:39.845+03,"{'time_on_page': '2.16', 'scroll_percent': 25}"
4,1753106952170378793,scroll_depth,2025-07-21 19:13:40.132+03,"{'time_on_page': '2.45', 'scroll_percent': 50}"
...,...,...,...,...
4953,1753522482590609787,click,2025-07-26 12:35:30.431+03,"{'element': {'id': None, 'tag': 'DIV', 'name':..."
4954,1753522482590609787,click,2025-07-26 12:35:32.277+03,"{'element': {'id': None, 'tag': 'DIV', 'name':..."
4955,1753522482590609787,scroll_depth,2025-07-26 12:35:50.755+03,"{'time_on_page': '67.06', 'scroll_percent': 50}"
4956,1753522482590609787,scroll_depth,2025-07-26 12:36:17.684+03,"{'time_on_page': '93.99', 'scroll_percent': 75}"


In [55]:
df[df['client_id'] == df.iloc[1302]['client_id']]

,client_id,event_type,timestamp,data
1061,1753208235896442169,click,2025-07-22 21:17:34.954+03,"{'element': {'id': None, 'tag': 'DIV', 'name':..."
1099,1753208235896442169,scroll_depth,2025-07-22 21:17:49.77+03,"{'time_on_page': '34.27', 'scroll_percent': 50}"
1119,1753208235896442169,scroll_depth,2025-07-22 21:18:02.502+03,"{'time_on_page': '47.00', 'scroll_percent': 75}"
1120,1753208235896442169,scroll_depth,2025-07-22 21:18:04.082+03,"{'time_on_page': '48.58', 'scroll_percent': 90}"
1300,1753208235896442169,session_start,2025-07-22 21:17:15.504+03,{'page_url': 'https://stirka.com/fr?utm_source...
1301,1753208235896442169,scroll_depth,2025-07-22 21:17:22.924+03,"{'time_on_page': '7.42', 'scroll_percent': 25}"
1302,1753208235896442169,first_interaction,2025-07-22 21:17:27.622+03,{'time_to_first_click': '12.12'}
1303,1753208235896442169,click,2025-07-22 21:17:27.622+03,"{'element': {'id': None, 'tag': 'DIV', 'name':..."


In [56]:
test = df_new[df_new['client_id'] == 1753522482590609787]
test


,client_id,event_type,timestamp,page_url,referrer,utm_params,device_info,time_on_page,scroll_percent,time_to_first_click,...,activity_summary,max_scroll_depth,session_duration,field_id,edit_count,field_type,time_spent,was_changed,hover_duration_ms,focus_lost_duration_ms
1917,1753522482590609787,first_interaction,2025-07-26 12:35:30.431+03,NaN,NaN,NaN,NaN,NaN,NaN,46.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4951,1753522482590609787,session_start,2025-07-26 12:34:43.698+03,https://stirka.com/fr?profit=450000&utm_source...,,"{'gclid': None, 'yclid': '4884201998581497855'...","{'language': 'ru-RU', 'user_agent': 'Mozilla/5...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4952,1753522482590609787,scroll_depth,2025-07-26 12:34:56.083+03,NaN,NaN,NaN,NaN,12.38,25.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4953,1753522482590609787,click,2025-07-26 12:35:30.431+03,NaN,NaN,NaN,NaN,46.73,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4954,1753522482590609787,click,2025-07-26 12:35:32.277+03,NaN,NaN,NaN,NaN,48.58,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4955,1753522482590609787,scroll_depth,2025-07-26 12:35:50.755+03,NaN,NaN,NaN,NaN,67.06,50.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4956,1753522482590609787,scroll_depth,2025-07-26 12:36:17.684+03,NaN,NaN,NaN,NaN,93.99,75.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4957,1753522482590609787,scroll_depth,2025-07-26 12:36:30.893+03,NaN,NaN,NaN,NaN,107.19,90.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
with open('conv_id.txt', 'r') as file:
    conv_id = file.read()  
conv_id = conv_id.split()


In [79]:
print([df[df['client_id'] == i] for i in conv_id])

[Empty DataFrame
Columns: [client_id, event_type, timestamp, data]
Index: [], Empty DataFrame
Columns: [client_id, event_type, timestamp, data]
Index: [], Empty DataFrame
Columns: [client_id, event_type, timestamp, data]
Index: [], Empty DataFrame
Columns: [client_id, event_type, timestamp, data]
Index: [], Empty DataFrame
Columns: [client_id, event_type, timestamp, data]
Index: [], Empty DataFrame
Columns: [client_id, event_type, timestamp, data]
Index: [], Empty DataFrame
Columns: [client_id, event_type, timestamp, data]
Index: [], Empty DataFrame
Columns: [client_id, event_type, timestamp, data]
Index: [], Empty DataFrame
Columns: [client_id, event_type, timestamp, data]
Index: [], Empty DataFrame
Columns: [client_id, event_type, timestamp, data]
Index: [], Empty DataFrame
Columns: [client_id, event_type, timestamp, data]
Index: [], Empty DataFrame
Columns: [client_id, event_type, timestamp, data]
Index: [], Empty DataFrame
Columns: [client_id, event_type, timestamp, data]
Index: []